In [1]:
#Basic
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import gc
import datetime as dt
from datetime import datetime, timedelta 
from openpyxl import load_workbook
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn import preprocessing, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_squared_error
import dask.dataframe as dd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import seaborn as sns
import collections as co
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from ipywidgets import widgets, interactive
from typing import Union
from tqdm.notebook import tqdm_notebook as tqdm
from itertools import cycle
from fastprogress import master_bar, progress_bar

pd.set_option('max_columns', 50)
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
current_path = os.getcwd()

def Set_Var(Choose_num_of_first_output_parameter, Choose_num_of_second_output_parameter, use_or_not_ProdBOE, use_or_not_DailyRate, use_or_not_CumProdBOE, 
use_or_not_OilProd, use_or_not_OilRate, use_or_not_CumOil, use_or_not_GasProd, use_or_not_GasRate, use_or_not_CumGas, 
use_or_not_WaterProd, use_or_not_CumMonth, use_or_not_ProdDays, use_or_not_CumProdDay, use_or_not_Shutin, use_or_not_Refrac):

    num_of_features = use_or_not_ProdBOE + use_or_not_DailyRate + use_or_not_CumProdBOE + use_or_not_OilProd + use_or_not_OilRate + use_or_not_CumOil + use_or_not_GasProd + use_or_not_GasRate + use_or_not_CumGas + use_or_not_WaterProd + use_or_not_CumMonth + use_or_not_ProdDays + use_or_not_CumProdDay + use_or_not_Shutin + use_or_not_Refrac
    tmp_1 = [use_or_not_ProdBOE, use_or_not_DailyRate, use_or_not_CumProdBOE, use_or_not_OilProd, use_or_not_OilRate, use_or_not_CumOil, use_or_not_GasProd, use_or_not_GasRate, use_or_not_CumGas, use_or_not_WaterProd, use_or_not_CumMonth, use_or_not_ProdDays, use_or_not_CumProdDay, use_or_not_Shutin, use_or_not_Refrac]
    tmp_2 = ["Prod_BOE", "ProdRate_BOE", "CumProd_BOE", 'LiquidsProd_BBL', 'Liq_rate', 'CumLiquids_BBL', 'GasProd_MCF', 'Gas_rate','CumGas_MCF', 'WaterProd_BBL', "TotalProdMonths", "ProducingDays", 'CumProdDay', "ShutinMonths", "Refrac"]
    used_features = []

    used_features = np.append(used_features, tmp_2[Choose_num_of_first_output_parameter - 1])
    used_features = np.append(used_features, tmp_2[Choose_num_of_second_output_parameter - 1])

    tmp_1 = np.delete(tmp_1, Choose_num_of_first_output_parameter - 1)
    tmp_2 = np.delete(tmp_2, Choose_num_of_first_output_parameter - 1)

    tmp_1 = np.delete(tmp_1, Choose_num_of_second_output_parameter - 2)
    tmp_2 = np.delete(tmp_2, Choose_num_of_second_output_parameter - 2)

    for idx, val in enumerate(tmp_1):
        if val == 1:
            used_features = np.append(used_features, tmp_2[idx])

    name_of_used_features = ''
    for name in used_features:
        name_of_used_features = name_of_used_features + '_' + name
    
    print("Input features =", used_features) 
    print("Onput features =", used_features[0], used_features[1])

    return used_features, name_of_used_features, num_of_features
    

    if not os.path.exists(current_path + '\\Model'):
        os.makedirs(current_path + '\\Model')

###  This function creates a sliding window or sequences of 28 days and one day label ####
###  For Multiple features                                                            ####
# def sliding_windows_mutli_features(data, seq_length):
#     x = []
#     y = []
#     for i in range((data.shape[0])-seq_length-1):
#         _x = data[i:(i+seq_length), :] ## 3 columns for features  
#         _y = data[i+seq_length, 0:2] ## column 0 contains the labbel
#         x.append(_x)
#         y.append(_y)

#     return np.array(x), np.array(y).reshape(-1, 2)

###  This function creates a sliding window or sequences of 28 days and one day label ####
###  For Multiple features                                                            ####
def sliding_windows_mutli_features(data, seq_length):
    x = []
    y = []
    for i in range((data.shape[0])-seq_length):
        _x_prod = data[i:(i+seq_length), 0:2] ## 3 columns for features
        _x_oper = data[i+1:(i+seq_length+1), 2:]
        _x = np.concatenate((_x_prod, _x_oper), axis=1)  
        _y = data[i+seq_length, 0:2] ## column 0 contains the labbel
        x.append(_x)
        y.append(_y)

    return np.array(x), np.array(y).reshape(-1, 2)

class LSTM2(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM2, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.batch_size = 1
        #self.seq_length = seq_length
        
        self.LSTM2 = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout = 0.2)
    
        self.fc1 = nn.Linear(hidden_size, 256)
        self.bn1 = nn.BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.dp1 = nn.Dropout(0.25)
        
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.dp2 = nn.Dropout(0.2)

        self.fc3= nn.Linear(128, 2)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        h_1 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size).to(device))
        c_1 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size).to(device))
        _, (hn, cn) = self.LSTM2(x, (h_1, c_1))
    
        #print("hidden state shpe is:",hn.size())
        y = hn.view(-1, self.hidden_size)
        
        final_state = hn.view(self.num_layers, x.size(0), self.hidden_size)[-1]
        #print("final state shape is:",final_state.shape)
        
        x0 = self.fc1(final_state)
        x0 = self.bn1(x0)
        x0 = self.dp1(x0)
        x0 = self.relu(x0)
        
        x0 = self.fc2(x0)
        x0 = self.bn2(x0)
        x0 = self.dp2(x0)
        x0 = self.relu(x0)
        
        out = self.fc3(x0)
        #print(out.size())
        return out

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x_data = x
        self.y_data = y

    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = Variable(torch.Tensor(self.x_data[idx]))
        y = Variable(torch.Tensor(self.y_data[idx]))
        return x, y


In [2]:
########################### Start of Control Pannel ###########################
"""
    - It should be same with the number of num_features and sum of use_or_not_features.
    - Ex. use_or_not_CumMonth = 1 means you will use that parameter.
"""
use_or_not_ProdBOE    = 0    # Ex. 3000, 1000, 500, ...
use_or_not_DailyRate  = 0    # Ex. 100, 90, 40, ...  = ProdBOE/ProdDays
use_or_not_CumProdBOE = 0    # Ex. 3000, 4000, 4500, ...
use_or_not_OilProd    = 0
use_or_not_OilRate    = 0
use_or_not_CumOil     = 0
use_or_not_GasProd    = 1
use_or_not_GasRate    = 0
use_or_not_CumGas     = 1
use_or_not_WaterProd  = 0 

use_or_not_CumMonth   = 0    # Ex. 1, 2, 3, 4, ...
use_or_not_ProdDays   = 1    # Ex. 30, 31, 29, 5, ...
use_or_not_CumProdDay = 1      
use_or_not_Shutin     = 1    # Ex. 0, 1, 0, 0, 2, 0, ...
use_or_not_Refrac     = 1

Choose_num_of_output_parameter = 2
Choose_num_of_first_output_parameter = 7
Choose_num_of_second_output_parameter = 9

########################### End of Control Pannel ###########################

used_features, name_of_used_features, num_of_features = Set_Var(Choose_num_of_first_output_parameter, Choose_num_of_second_output_parameter, use_or_not_ProdBOE, use_or_not_DailyRate, use_or_not_CumProdBOE, 
                                                                            use_or_not_OilProd, use_or_not_OilRate, use_or_not_CumOil, use_or_not_GasProd, use_or_not_GasRate, use_or_not_CumGas, 
                                                                            use_or_not_WaterProd, use_or_not_CumMonth, use_or_not_ProdDays, use_or_not_CumProdDay, use_or_not_Shutin, use_or_not_Refrac)

Input features = ['GasProd_MCF' 'CumGas_MCF' 'ProducingDays' 'CumProdDay' 'ShutinMonths'
 'Refrac']
Onput features = GasProd_MCF CumGas_MCF


In [3]:
use_datetime = 'BaseCase' # Check the data in the Data dictionary.
seq_length = 3
lamda = 0 ### 생산량과 누적생산량 맞추기 비율, 앞에 lamda 붙어있음 (확인 다시 필요)
choose_scaler = 1 # 1=standard, 2=minmax, 3=Robust, 4=QuantileTransformer, 5=PowerTransformer
num_of_out_features = 2
seed = 1
############################## Hyperparameters ##############################
batch_size = 128 # 128
num_epochs = 100
learning_rate = 1e-2 # 1e-3
hidden_size = 128
num_layers = 3
num_classes = 1
best_val_loss = 1

In [59]:
"""
Train / Valid / Test Dataset 만들기, Refrac 데이터도 각 비율 만큼 넣기
"""
input_path_train = current_path + '\\Data\\After_Preprocess\\' + use_datetime + '\\Pre_Train\\'
pre_train_file_list = os.listdir(input_path_train)

input_path_train_refrac = current_path + '\\Data\\After_Preprocess\\' + use_datetime + '\\Pre_Refrac\\Train\\'
pre_train_file_list_refrac = os.listdir(input_path_train_refrac)

if not os.path.exists(current_path + '\\Data\\After_Preprocess\\' + use_datetime + '\\Train\\'):
        os.makedirs(current_path + '\\Data\\After_Preprocess\\' + use_datetime + '\\Train\\')

if not os.path.exists(current_path + '\\Data\\After_Preprocess\\' + use_datetime + '\\Valid\\'):
        os.makedirs(current_path + '\\Data\\After_Preprocess\\' + use_datetime + '\\Valid\\')

train_input_path = current_path + '\\Data\\After_Preprocess\\' + use_datetime + '\\Train\\'
valid_input_path = current_path + '\\Data\\After_Preprocess\\' + use_datetime + '\\Valid\\'

seed = 2
np.random.seed(seed)
np.random.shuffle(pre_train_file_list)
np.random.shuffle(pre_train_file_list_refrac)

train_size = int(len(pre_train_file_list) * 0.8235)
train_size_refrac = int(len(pre_train_file_list_refrac) * 0.8235)

for file in pre_train_file_list[0:train_size]:
    df = pd.read_csv(input_path_train + file)
    df.to_csv(train_input_path + file)

for file in pre_train_file_list[train_size:]:
    df = pd.read_csv(input_path_train + file)
    df.to_csv(valid_input_path + file)   

for file in pre_train_file_list_refrac[0:train_size_refrac]:
    df = pd.read_csv(input_path_train_refrac + file)
    df.to_csv(train_input_path + file)

for file in pre_train_file_list_refrac[train_size_refrac:]:
    df = pd.read_csv(input_path_train_refrac + file)
    df.to_csv(valid_input_path + file)

In [25]:
"""
Scale을 Feature 단위로 할 때
"""
# df_tot_tmp = pd.DataFrame()
# df_tot = pd.DataFrame(columns=used_features)
# for file in scale_input_list:
#     df_tmp = pd.read_csv(scale_input_path + file)
#     df_tot_tmp = pd.concat([df_tot_tmp, df_tmp])

# print(df_tmp.shape)
# print(df_tot_tmp.shape)

# for x in used_features:
#     df_tot[x] = df_tot_tmp[x]
# data_for_scale = np.array(df_tot)

# print(len(used_features), used_features)
# display(df_tot.head(10))
# display(data_for_scale)

In [26]:
used_features

array(['GasProd_MCF', 'CumGas_MCF', 'ProducingDays', 'CumProdDay',
       'ShutinMonths', 'Refrac'], dtype='<U32')

In [217]:
scaler_prod = MinMaxScaler(feature_range=(0, 1))
scaler_cumprod = MinMaxScaler(feature_range=(0, 0.0375))
scaler_oper = MinMaxScaler(feature_range=(0, 1))

In [207]:
def ReturnDataLoader(path, filename, used_seq):
    df_temp = pd.read_csv(path + filename)
    df = pd.DataFrame(columns=used_features)
    for x in used_features:
        df[x] = df_temp[[x]]
    data = np.array(df)
    scaler_prod.fit(data[:used_seq, 0].reshape(-1, 1))
    scaler_cumprod.fit(data[:used_seq, 1].reshape(-1, 1))
    scaler_oper.fit(data[:, 2:])

    scaled_prod = scaler_prod.transform(data[:, 0].reshape(-1, 1))
    scaled_cumprod = scaler_cumprod.transform(data[:, 1].reshape(-1, 1))
    scaled_oper = scaler_oper.transform(data[:, 2:])

    train_data_scaled = np.concatenate((scaled_prod, scaled_cumprod), axis=1)
    train_data_scaled = np.concatenate((train_data_scaled, scaled_oper), axis=1)

    x, y = sliding_windows_mutli_features(train_data_scaled, seq_length)
    trainX = torch.Tensor(x)
    trainY = torch.Tensor(y)
    train_dataset = CustomDataset(trainX, trainY)
    batch_size = int(len(trainX))
    dataloader = DataLoader(train_dataset, batch_size, shuffle=False)

    return dataloader

def Cal_Restriction(outputs):
    t_2 = (outputs[:, 1].cpu().detach().numpy())
    t_1 = np.zeros(len(t_2))
    t_1[1:] = t_2[0:-1]
    zero_ = np.zeros(len(t_2))
    pred_ = t_2 - t_1
    res_lst = np.minimum(zero_, pred_)
    restriction = (np.sum(np.abs(res_lst)))

    return restriction

In [218]:
used_seq = 3
well_len_lst = []
best_val_loss = 50000
X = []
Y = []
loss_lst = []
input_size = num_of_features
lstm = LSTM2(num_classes, input_size, hidden_size, num_layers)
lstm.to(device)

lstm.apply(init_weights)

criterion = torch.nn.MSELoss().to(device)    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=50, factor =0.5, min_lr=1e-7, eps=1e-08)

lamda = 1
num_epochs = 100

train_input_path = current_path + '\\Data\\After_Preprocess\\' + use_datetime + '\\Train\\'
train_file_list = os.listdir(train_input_path)

valid_input_path = current_path + '\\Data\\After_Preprocess\\' + use_datetime + '\\Valid\\'
valid_file_list = os.listdir(valid_input_path)

for epoch in progress_bar(range(num_epochs)):
    np.random.shuffle(train_file_list)
    loss_lst = []
    val_loss_lst = []
    lstm.train()
    for file in train_file_list:
        train_dataloader = ReturnDataLoader(train_input_path, file, used_seq)
        for data in train_dataloader:
            x_train, y_train = data
            outputs = lstm(x_train.to(device))
            optimizer.zero_grad()
            torch.nn.utils.clip_grad_norm_(lstm.parameters(), 1)
            # restriction = Cal_Restriction(outputs)
            # loss = lamda * criterion(outputs[:, 0], y_train[:, 0].to(device)) + (1-lamda) * criterion(outputs[:, 1], y_train[:, 1].to(device)) + restriction
            loss = lamda * criterion(outputs[:, 0], y_train[:, 0].to(device)) + (1-lamda) * criterion(outputs[:, 1], y_train[:, 1].to(device))
            loss_lst = np.append(loss_lst, loss.cpu().item())
            loss.backward()
            optimizer.step()

    train_loss_tot = torch.tensor(loss).to(device)

    with torch.no_grad():
        lstm.eval()
        for file_ in valid_file_list:
            valid_dataloader = ReturnDataLoader(valid_input_path, file_, used_seq)
            for data_ in valid_dataloader:
                x_valid, y_valid = data_
                outputs_ = lstm(x_valid.to(device))
                # restriction_ = Cal_Restriction(outputs_)
                # val_loss = lamda * criterion(outputs_[:, 0], y_valid[:, 0].to(device)) + (1-lamda) * criterion(outputs_[:, 1], y_valid[:, 1].to(device)) + restriction_
                val_loss = lamda * criterion(outputs_[:, 0], y_valid[:, 0].to(device)) + (1-lamda) * criterion(outputs_[:, 1], y_valid[:, 1].to(device))

                val_loss_lst = np.append(val_loss_lst, val_loss.cpu().item())

        val_loss_tot = torch.tensor(np.mean(val_loss_lst)).to(device)

        if val_loss_tot.cpu().item() < best_val_loss:
            torch.save(lstm.state_dict(), current_path + '\\Model\\Best_model_Case_' + name_of_used_features + '_' + str(seq_length) + 'seq_' + use_datetime + '_Test' + '.pt')
            print("Saved best model epoch:", epoch, "val loss is:", val_loss_tot.cpu().item(), "train loss is", train_loss_tot.cpu().item())
            best_val_loss = val_loss_tot.cpu().item()

        if epoch % 1 == 0:
            print("Epoch: %d, loss: %1.5f valid loss:  %1.5f " %(epoch, train_loss_tot.cpu().item(), val_loss_tot.cpu().item()))

        scheduler.step(val_loss)



Saved best model epoch: 0 val loss is: 26518.35540607248 train loss is 4.5549774169921875
Epoch: 0, loss: 4.55498 valid loss:  26518.35541 
Saved best model epoch: 1 val loss is: 26494.411539974502 train loss is 2.238518476486206
Epoch: 1, loss: 2.23852 valid loss:  26494.41154 
Epoch: 2, loss: 4.43988 valid loss:  26578.67015 
Saved best model epoch: 3 val loss is: 26492.61336929764 train loss is 4.579494953155518
Epoch: 3, loss: 4.57949 valid loss:  26492.61337 
Saved best model epoch: 4 val loss is: 25934.172495182116 train loss is 2.0550918579101562
Epoch: 4, loss: 2.05509 valid loss:  25934.17250 
Epoch: 5, loss: 1.56111 valid loss:  25950.88703 
Epoch: 6, loss: 1.19161 valid loss:  26572.51952 
Saved best model epoch: 7 val loss is: 25916.932486341822 train loss is 1.9410923719406128
Epoch: 7, loss: 1.94109 valid loss:  25916.93249 
Epoch: 8, loss: 0.42165 valid loss:  26526.96840 
Saved best model epoch: 9 val loss is: 24764.779407531023 train loss is 4.697839736938477
Epoch: 9,

KeyboardInterrupt: 